In [393]:
import classla
import pandas as pd
import numpy as np
import re
import string
#classla.download('hr')
#classla.download('hr', type='nonstandard')

In [394]:
pos_tags = "ADJ – ADP – ADV – AUX – CCONJ – DET – INTJ – NOUN – NUM – PART – PRON – PROPN – PUNCT – SCONJ – SYM – VERB – X".split(" – ")
relations = "acl – advcl – advmod – advmod:emph – amod – appos – aux – case – cc – ccomp – compound – conj – cop – csubj – dep – det – det:numgov – discourse – dislocated – expl – fixed – flat – flat:foreign – iobj – list – mark – nmod – nsubj – nummod – obj – obl – orphan – parataxis – punct – root – vocative – xcomp".split(" – ")
word_features = "Animacy – Case – Definite – Degree – Foreign – Gender – Gender[psor] – Mood – Number – Number[psor] – NumType – Person – Polarity – Poss – PronType – Reflex – Tense – VerbForm – Voice".split(" – ")

In [395]:
def is_float(element):
    try:
        float(element)
        return True
    except ValueError:
        return False

In [396]:
artist_name = "Severina"
df = pd.read_csv(f'data/{artist_name}_final.csv')



In [397]:
df.Lyrics_lyricstranslate.isnull().sum()

7

In [398]:
df.Lyrics_lyricstranslate.fillna(df.Lyrics_cuspajz).isnull().sum()


3

In [399]:
df.Lyrics_lyricstranslate.fillna(df.Lyrics_cuspajz).fillna(df.Lyrics_tekstovinet).isnull().sum()

0

In [400]:
lyrics_df = df.Lyrics_lyricstranslate.fillna(df.Lyrics_cuspajz).fillna(df.Lyrics_tekstovinet)

In [401]:
song_df = df.Song_lyricstranslate.fillna(df.Song_cuspajz).fillna(df.Song_tekstovinet)

In [402]:
refren = re.compile(('refren[:\)\]\.]'), re.IGNORECASE)
refrain = re.compile(('refrain[:\)\]\.]'), re.IGNORECASE)
pripjev = re.compile(('pripjev[:\)\]\.]'), re.IGNORECASE)

ref = re.compile('ref[\.,:\ )\]]', re.IGNORECASE)  #ref followed by any of chars in braacket
ref_last = re.compile('ref$', re.IGNORECASE)  #ref followed by any of chars in braacket


In [403]:
lyrics = []
for song_lyrics in lyrics_df:
    song_lyrics_corrected = []
    for l in song_lyrics.splitlines():
        l = refren.sub("", l)
        l = refrain.sub("", l)
        l = ref.sub("", l)
        l = ref_last.sub("", l)
        l = pripjev.sub("", l)
        l = re.sub("Refren", "", l)  #remove word Refren (if its uppercase) 
        l = re.sub("Refrain", "", l)  #remove word Refrain (if its uppercase)         
        l = re.sub("Pripjev", "", l)  #remove word Pripjev (if its uppercase)     

        l = re.sub("\dx", "", l)    #remove \digit x (usually left after ref)
        l = re.sub("x\d", "", l)    #remove x \digit
        
        l = re.sub("()", "", l)  #remove brackets without content
        l = re.sub("\[\]", "", l)  

        l = l.strip()
        song_lyrics_corrected.append(l)
        
    lyrics.append([l for l in song_lyrics_corrected if not is_float(l) and "http" not in l and len(l)>0])

In [404]:
all_lyrics = ["".join([(". " if c[0].isupper() else ", ")+c for c in l]).strip() for l in lyrics]

In [405]:
corrected_lyrics=[]
for l in all_lyrics:

    l.strip()
    l = re.sub(f" [{string.punctuation}] ", " ", l)      #remove special characters that are flanked by empty spaces on both sides
    l = re.sub(f" [{string.punctuation}]", " ", l)      #remove special characters that are flanked by empty spaces on left

    l = re.sub(",,", ",", l)    #replace two commas with one
    l = re.sub("\.,", ".", l)    #replace dot comma with dot
    l = re.sub(",\.", ",", l)    #replace comma dot with comma
    l = re.sub("(?<!\.)\.\.(?!\.)", ".", l)    #replace two dots (that are neither preceded nor followed by third dot), with single dot
    l = re.sub("\.\.\.\.[\.]*", "...", l)    #replace anything with more than three dots with three dots
    l.strip()

    l=l[2:] #remove the dot that was left from the concetination
    corrected_lyrics.append(l)


In [406]:
for l in corrected_lyrics:
    print(l)
    print("#####")
    print("#####")

Falit ćeš mi, faliti, ali neću žaliti, sanjala sam, sanjala, da ću sresti pravog tigra, lava. Boljeti će, boljeti, teško ću preboljeti, radije ću patiti, nego ti se vratiti. Nerviraš me k'o komarac, velik' bumbar, mali žalac, ne dam pusu, na ti bonu, da te lupi po hormonu. Ringe raja, eci, pec, nisi tigar, ti si zec. Ko to pita, ko to pita, s' kim to moje srce skita, ajde lave izađi, pa mi srce pronađi, da mi suknja poludi, da mi pjesma puni grudi. Ko to pita, ko to pita, s' kim to moje srce skita, ajde lave izađi, pa mi srce pronađi, da mi suknja poludi, da mi pjesma, da mi pjesma puni grudi. Falit ćeš mi, faliti, neću, neću zaliti, sanjala sam, sanjala, da ću sresti pravog tigra, lava. Nerviraš me k'o komarac, velik' bumbar, mali žalac, nije meni do slatkiša, hoću lava, neću miša. Ringe raja, ćevap šiš, nisi lav, ti si miš. Eci, pec, zvrc, zvrc. Op, op, op. Ajde, ajde, ajde, ajde. Ćevap šiš. Ringe raja, ćevap šiš, ja sam lav, ti si miš
#####
#####
Verse 1]. Stvorio te Bog samo mog da

In [407]:
final_lyrics_df = pd.DataFrame([df.Song_ID.values, song_df.values, corrected_lyrics]).transpose()
final_lyrics_df.columns = ["Song_ID", "Song", "Lyrics"]
final_lyrics_df
final_lyrics_df.to_csv(f"data/{artist_name}_final_lyrics.csv")

In [408]:
#classla.download('hr', type='nonstandard')
nlp = classla.Pipeline('hr', type='nonstandard') # run classla.download('hr', type='nonstandard') beforehand if necessary

2022-02-21 18:55:47 INFO: Loading these models for language: hr (Croatian):
| Processor | Package     |
---------------------------
| tokenize  | nonstandard |
| pos       | nonstandard |
| lemma     | nonstandard |
| depparse  | standard    |
| ner       | nonstandard |

2022-02-21 18:55:47 INFO: Use device: cpu
2022-02-21 18:55:47 INFO: Loading: tokenize
2022-02-21 18:55:47 INFO: Loading: pos
2022-02-21 18:55:48 INFO: Loading: lemma
2022-02-21 18:55:54 INFO: Loading: depparse
2022-02-21 18:55:55 INFO: Loading: ner
2022-02-21 18:55:55 INFO: Done loading processors!


In [409]:
df_nlp = pd.DataFrame()

for cl, song, song_id in zip(corrected_lyrics, song_df, df["Song_ID"].values):

    doc = nlp(cl)
    doc_dict = doc.to_dict()

    #df_song = pd.DataFrame([doc_dict[i][0] for i, _ in enumerate(doc_dict)]).transpose()
    sentences = [doc_dict[i][0] for i, _ in enumerate(doc_dict)]
    df_song = pd.DataFrame(sentences[0])
    for sentence in sentences[1:]:
        df_song = pd.concat([df_song, pd.DataFrame(sentence)])

    df_song["Song"] = song 
    df_song["Song_ID"] = song_id
    df_nlp = pd.concat([df_nlp, df_song])

In [410]:
df_nlp.reset_index(drop=True, inplace=True)

In [411]:
features_df = pd.DataFrame()
rows = df_nlp.feats.str.split("|").to_list()

In [412]:
for i, row in enumerate(rows):
    if not type(row) == float:
        for f in row:
            features_df.loc[i, f[0:f.index("=")]] = f[f.index("=")+1:]


In [413]:
df_nlp = df_nlp.join(features_df)

In [414]:
df_nlp.to_csv(f"{artist_name}_nlp.csv", index=False)

In [415]:
df_nlp[(df_nlp.ner=="B-LOC")]["lemma"].value_counts()[0:10]

Marjan      4
Split       4
Brazil      2
Azija       2
Kina        2
Rio         1
Zagreb      1
južan       1
vatren      1
Hrvatska    1
Name: lemma, dtype: int64

In [416]:
df_nlp[(df_nlp.ner=="B-ORG")]["lemma"].value_counts()[0:10]

Tuborg      2
Calimero    1
Hajduk      1
Manolov     1
Diorov      1
Viber       1
Name: lemma, dtype: int64

In [417]:
df_nlp[(df_nlp.ner=="B-PER")]["lemma"].value_counts()[0:50]

Bog            36
Marija          5
Bogo            5
Marijo          5
Ante            5
Severina        4
Hurem           3
Lola            3
Taliman         3
Esmeralda       3
Croatia         2
Split           2
Buldog          2
Dioklecijan     2
božiji          2
Hrvatica        2
Duje            2
Jadran          2
Isus            2
svet            2
Mare            2
Cartier         2
Marjan          2
Jere            1
Danica          1
andiamati       1
Alcatraz        1
Seve            1
Copacabana      1
Brad            1
Fića            1
Adame           1
Nagi            1
drug            1
haljinica       1
Frane           1
Šime            1
šic             1
Calimero        1
Muškarec        1
Tony            1
Name: lemma, dtype: int64

In [418]:
df_nlp[(df_nlp.upos=="SYM")]["lemma"].value_counts()[0:10]

Series([], Name: lemma, dtype: int64)

In [419]:
df_nlp[(df_nlp.upos=="PROPN")]["lemma"].value_counts()[0:10]

Bog           36
Verse         11
Marjan         8
Split          6
Marijo         6
Bogo           5
Beo            5
Pre-Chorus     5
Marija         5
Ante           5
Name: lemma, dtype: int64

In [420]:
df_nlp[(df_nlp.upos=="PROPN")& (df_nlp.ner=="B-PER")]["lemma"].value_counts()[0:10]

Bog          36
Marija        5
Marijo        5
Bogo          5
Ante          5
Severina      4
Hurem         3
Lola          3
Taliman       3
Esmeralda     3
Name: lemma, dtype: int64

In [421]:
df_nlp[df_nlp.lemma=="Goran"].Song.unique()

array([], dtype=object)

In [422]:
df_nlp[(df_nlp.upos=="VERB")]["lemma"].value_counts()[0:10]

znati      141
voljeti    107
dati       101
imati       93
ljubiti     83
nemati      78
moći        61
htjeti      51
ići         47
trebati     45
Name: lemma, dtype: int64

In [423]:
df_nlp[(df_nlp.upos=="NOUN")]["lemma"].value_counts()[0:10]

ljubav    134
srce      105
noć        70
oko        54
svijet     51
dan        45
ruka       38
duša       37
suza       37
vikend     36
Name: lemma, dtype: int64

In [424]:
df_nlp[(df_nlp.upos=="NOUN")]["lemma"].value_counts()[0:10]

ljubav    134
srce      105
noć        70
oko        54
svijet     51
dan        45
ruka       38
duša       37
suza       37
vikend     36
Name: lemma, dtype: int64

In [425]:
df_nlp[df_nlp.lemma=="Hajduk"].Song.unique()

array(['Slaba na slabića'], dtype=object)

In [426]:
df_nlp[df_nlp.lemma=="Hajduk"].Song_ID.unique()

array(['538Exp8s_1_120'], dtype=object)

In [427]:
df_nlp[df_nlp.lemma=="Indira"].Song_ID.unique()

array([], dtype=object)

In [428]:
df[df.Song_ID == '538Exp8s_1_120'].Lyrics_tekstovinet.values

array(['Nudili mi pola Novog Sada\r\nZagreb nocu kada kisa pada\r\nnudili mi Bosanske doline\r\npored Save zelja da me mine\r\n\nRef.\r\nA ja slaba na slabica\r\nsto suplja mu je prica\r\npa me laze evo noc i dan\r\n \nSlaba na slabica\r\nsto nema ni za pica\r\nali sta cu kad je lijep ko san\r\n\nAjmo momci\r\n\nNudili mi rasne konje vrane\r\ni bogatstvo da ti pamet stane\r\nnudili mi trideset salasa\r\nda me prati dvijesto tamburasa\r\n\nRef. \r\n\nNudili mi otoke kraj Splita\r\ni da Hajduk bude prvak svita\r\nnudili mi brda i doline\r\na reci Seve sta ti srce zine\r\n\nRef. 2x'],
      dtype=object)